In [1]:
from google.colab import drive
import pandas as pd

drive.mount('/content/drive')
train = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/train_cabin.csv')
test = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Datasets/test_cabin.csv')


Mounted at /content/drive


In [2]:
train1=train.copy()
test1 = test.copy()

In [3]:
test1 = test1.fillna(-1)

In [4]:
from sklearn.model_selection import train_test_split,cross_val_score,GridSearchCV
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score

import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier,VotingClassifier, AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier


# Machine Learning


The best model was GradientBoosting
 Kaggle = 0.78468 top 10%
 (the last model) 

# SVC

In [7]:
y = train1['Survived']
X = StandardScaler().fit_transform(train1[features])
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.5)

In [ ]:
param_grid= {'C':[0.00001,0.0001,0.001,0.01,0.1,1,5,10,20], 'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],'degree': range(2,5),'gamma':['scale', 'auto'],'probability': [True,False],}


svc_cv = GridSearchCV(SVC(), param_grid, cv=10).fit(X_train,y_train)


# Print the tuned parameters and score
print("SVC Parameters: {}".format(svc_cv.best_params_))
print("Best score is {}".format(svc_cv.best_score_))

SVC Parameters: {'C': 1, 'degree': 2, 'gamma': 'scale', 'kernel': 'rbf', 'probability': True}
Best score is 0.8360606060606062


In [8]:
svc = SVC(C=1, degree = 2, gamma= 'scale', kernel = 'rbf', probability = True).fit(X_train,y_train)


svc_cross = cross_val_score(svc, X_train,y_train, n_jobs=-1)

y_pred_svc =  svc.predict(X_test)

print('Cross validation Scores:\n', (svc_cross))
print('Cross validation Scores Mean:\n', (svc_cross.mean()))
print('Score test {:.4f}'.format(svc.score(X_test,y_test)))
print('Score train: {:.4f}'.format(svc.score(X_train,y_train)))
print(confusion_matrix(y_test, y_pred_svc))
print(classification_report(y_test, y_pred_svc))

Cross validation Scores:
 [0.83146067 0.82022472 0.78651685 0.85393258 0.87640449]
Cross validation Scores Mean:
 0.8337078651685393
Score test 0.8206
Score train: 0.8449
[[248  35]
 [ 45 118]]
              precision    recall  f1-score   support

           0       0.85      0.88      0.86       283
           1       0.77      0.72      0.75       163

    accuracy                           0.82       446
   macro avg       0.81      0.80      0.80       446
weighted avg       0.82      0.82      0.82       446



# RandomForestClassifier:

In [12]:
features_new3 = ['Sex_female','Name_Mr','Sex_male','Fare','Age','Pclass_A','Pclass_C','Cabin_X','Name_Miss','Name_Mrs','Family size_Small',
                'Family size_Huge','Family size_Alone','Embarked_S','Pclass_B','Embarked_C','Name_Master','Cabin_C','Family size_Large',
                'Cabin_B','Cabin_E','Cabin_D']
y = train1['Survived']
X= train1[features_new3]
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.5)

In [ ]:
# Setup the parameters and distributions to sample from: param_dist
param_dist = {"n_estimators": [150,200],
              "max_features": range(1,5),
              'max_depth': [4,5,6],
              "min_samples_leaf": [1,2,3,4,5,6],
              "criterion": ["gini", "entropy"]}


rt4 = RandomForestClassifier(n_jobs=-1)

rt4_cv = GridSearchCV(rt4, param_dist, cv=5, n_jobs=-1).fit(X_train,y_train)


# Print the tuned parameters and score
print("Tuned Random Forest Parameters: {}".format(rt4_cv.best_params_))
print("Best score is {}".format(rt4_cv.best_score_))

Tuned Random Forest Parameters: {'criterion': 'gini', 'max_depth': 6, 'max_features': 4, 'min_samples_leaf': 1, 'n_estimators': 150}
Best score is 0.8314606741573034


In [13]:
rf4 = RandomForestClassifier(criterion= 'gini', max_depth= 5,max_features= 1, min_samples_leaf= 1, n_estimators= 150, random_state=0).fit(X_train,y_train)

rf4_cross = cross_val_score(rf4, X_train,y_train, n_jobs=-1)

y_pred_rf4 =  rf4.predict(X_test)

print('Cross validation Scores:\n', (rf4_cross))
print('Cross validation Scores Mean:\n', (rf4_cross.mean()))
print('Score test {:.4f}'.format(rf4.score(X_test,y_test)))
print('Score train: {:.4f}'.format(rf4.score(X_train,y_train)))
print('\n\n')
print(confusion_matrix(y_test, y_pred_rf4))
print(classification_report(y_test, y_pred_rf4))

Cross validation Scores:
 [0.80898876 0.80898876 0.78651685 0.84269663 0.84269663]
Cross validation Scores Mean:
 0.8179775280898877
Score test 0.8139
Score train: 0.8629



[[239  44]
 [ 39 124]]
              precision    recall  f1-score   support

           0       0.86      0.84      0.85       283
           1       0.74      0.76      0.75       163

    accuracy                           0.81       446
   macro avg       0.80      0.80      0.80       446
weighted avg       0.82      0.81      0.81       446



# GradientBoosting with more features:
- kaggle = 0.78468 top 10%

In [15]:
from sklearn.model_selection import train_test_split
features0 = ['Age', 'Fare', 'Name_Dona','Name_Capt', 'Name_Col', 'Name_Don', 'Name_Dr', 'Name_Jonkheer','Name_Lady', 'Name_Major', 'Name_Master',
            'Name_Miss', 'Name_Mlle','Name_Mme', 'Name_Mr', 'Name_Mrs', 'Name_Ms', 'Name_Rev', 'Name_Sir','Name_the Countess', 'Pclass_A', 
            'Pclass_B', 'Pclass_C', 'Sex_female','Sex_male', 'Embarked_C', 'Embarked_Q', 'Embarked_S','Family size_Alone', 'Family size_Huge', 
            'Family size_Large','Family size_Small', 'Cabin_A', 'Cabin_B', 'Cabin_C', 'Cabin_D','Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T', 
            'Cabin_X']
y = train1['Survived']
X= train1[features0]


X_train, X_test, y_train, y_test = train_test_split(X,y,random_state=0,test_size=0.5)

In [ ]:
param_dist = {'loss': ['desviance','exponential'],
              'learning_rate': [0.0085,0.0084,0.0083,0.0082],
              'min_samples_split':range(2,8),
              'min_samples_leaf':range(1,8),
              'max_depth':range(1,8)}

gb3 = GradientBoostingClassifier(random_state=0)

gb3_cv = GridSearchCV(gb3, param_dist, cv=10, n_jobs=-1).fit(X_train,y_train)

In [ ]:
print("Tuned GradientBoosting Parameters: {}".format(gb3_cv.best_params_))
print("Best score is {}".format(gb3_cv.best_score_))

Tuned Decision Tree Parameters: {'learning_rate': 0.0085, 'loss': 'exponential', 'max_depth': 4, 'min_samples_leaf': 2, 'min_samples_split': 3}

In [ ]:
gb3 = GradientBoostingClassifier(learning_rate=0.0085, loss= 'exponential', max_depth= 4, min_samples_leaf= 2, min_samples_split= 3).fit(X_train,y_train)

gb3_cross = cross_val_score(gb3,X_train,y_train, n_jobs=-1)
y_pred_gb3 = gb3.predict(X_test)

print('Cross validation Scores:\n', (gb3_cross))
print('Cross validation Scores Mean:\n', (gb3_cross.mean()))
print('Score test {:.4f}'.format(gb3.score(X_test,y_test)))
print('Score train: {:.4f}'.format(gb3.score(X_train,y_train)))
print('\n\n')
print(confusion_matrix(y_test, y_pred_gb3))
print(classification_report(y_test, y_pred_gb3))

In [ ]:
X_true = test1[features0]
X = train1[features0]
y= train1['Survived']

gb3_true = GradientBoostingClassifier(learning_rate=0.0085, loss= 'exponential', max_depth= 4, min_samples_leaf= 2, min_samples_split= 3).fit(X,y)

y_pred_gb3_true = gb3.predict(X_true)

In [ ]:
output = pd.DataFrame({'PassengerId': test1.PassengerId, 'Survived': y_pred_gb3_true})
output.to_csv('my_submission[gradientboost1].csv', index=False)
output

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
